In [141]:
# load the data set
import pandas as pd

PATH="C:/Users/Owner/Desktop/projects\ML/chapter_2/ml_scans_changed.csv"
scans=pd.read_csv(PATH)

#scans.info()
scans.iloc[202:205]

,325,335,345,355,5,15,25,35,45,direction,correction
202,364.0,NaN,295.0,264.0,288.0,276.0,231.0,NaN,86.0,36.0,0.0
203,363.0,294.0,298.0,263.0,288.0,271.0,232.0,NaN,NaN,38.0,0.0
204,361.0,295.0,289.0,260.0,289.0,279.0,233.0,106.0,87.0,40.0,0.0


In [142]:
# repalce 0 value to median
#df['a']=df['a'].replace(0,df['a'].mean())
cols = ['325'	,'335',	'345',	'355',	'5',	'15',	'25',	'35',	'45']
#scans[cols]=scans[cols].replace(0,scans[cols].mean(axis=1))

#scans[cols]=scans[cols].replace(0,scans[cols])
#scans.fillna(scans.mean(axis=1), axis=1)

#### replace nan with neighbour sum/2
m = scans[cols].mean(axis=1)
#m=30
for i, col in enumerate(scans[cols]):
    # using i allows for duplicate columns
    # inplace *may* not always work here, so IMO the next line is preferred
    # df.iloc[:, i].fillna(m, inplace=True)
    scans.iloc[:, i] = scans.iloc[:, i].fillna((scans.iloc[:, i+1]+scans.iloc[:, i-1])/2)
    
### if still has Nan== index -1 value
for i, col in enumerate(scans[cols]):
    scans.iloc[:, i] = scans.iloc[:, i].fillna(scans.iloc[:, i-1])

scans.iloc[202:205]

,325,335,345,355,5,15,25,35,45,direction,correction
202,364.0,329.5,295.0,264.0,288.0,276.0,231.0,158.5,86.0,36.0,0.0
203,363.0,294.0,298.0,263.0,288.0,271.0,232.0,232.0,232.0,38.0,0.0
204,361.0,295.0,289.0,260.0,289.0,279.0,233.0,106.0,87.0,40.0,0.0


In [143]:
# separate train and test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(scans, test_size=0.2, random_state=42)
#train_set.info()
#test_set.info()

In [144]:
## looking for correlation 
corr_matrix=scans.corr()
corr_matrix['correction'].sort_values(ascending=False)

correction    1.000000
345           0.273351
325           0.263671
335           0.215391
355           0.095328
5             0.052156
15           -0.104242
25           -0.162764
35           -0.311255
45           -0.322883
direction    -0.438641
Name: correction, dtype: float64

In [145]:
############## prepare the data for ML(separate labels from predictors)
train_labels = train_set["correction"].copy() # labels
train = train_set.drop("correction", axis=1) # predictors(witout lables)
scans.iloc[:5]
scans_labels.iloc[:5]

331   -110.0
409   -130.0
76       0.0
868      0.0
138      0.0
Name: correction, dtype: float64

In [146]:
#scans.info()

In [147]:
"""Training and Evaluating on the Training Set"""
# linear Regression
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train, train_labels) # prepared data(parameters) and labels

LinearRegression()

In [148]:
# Test linear model
some_data = train.iloc[40:45]
some_labels = train_labels.iloc[40:45]


print("Predictions:", lin_reg.predict(some_data))
print("Labels:", list(some_labels))


Predictions: [  20.4056397519  -29.0768043806  -41.9655963447   33.9570855455
 -131.3026401908]
Labels: [0.0, 0.0, 0.0, 0.0, -170.0]


In [149]:
"""DecisionTreeRegressor. This is a powerful model, capable of finding
complex nonlinear relationships in the data"""
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(train, train_labels)

DecisionTreeRegressor()

In [152]:
f_i=500
l_i=510
some_data = train.iloc[f_i:l_i]
some_labels = train_labels.iloc[f_i:l_i]

print("Predictions:", tree_reg.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [   0.   10.    0. -155.  125.    0.  116.    0.    0.    0.]
Labels: [0.0, 10.0, 0.0, -155.0, 125.0, 0.0, 116.0, 0.0, 0.0, 0.0]


In [153]:
from sklearn.model_selection import cross_val_score
import numpy as np
np.set_printoptions(precision=10, suppress=True)

scores = cross_val_score(tree_reg, train, train_labels,
scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
print(tree_rmse_scores)

[39.3992650101 19.93766327   43.9080478576 36.5055646443 32.4777167198
 43.7880786668 20.8244147577 31.3566513412 12.8496919548 36.3209076612]


In [154]:
''' Evaluate Your System on the Test Set '''
from sklearn.metrics import mean_squared_error
y_test = test_set["correction"].copy()
X_test = test_set.drop("correction", axis=1)

final_predictions = tree_reg.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_rmse) # => evaluates to 

38.05171919024597


In [155]:
# Look of results from Cross Validation for DecisionTreeRegressor
def display_scores(scores):
    print("Scores:", scores) # errors in every 10 small sets
    print("Mean:", scores.mean()) # estimate of the performance of your model
    print("Standard deviation:", scores.std()) # precise this estimate
    
display_scores(tree_rmse_scores)

Scores: [39.3992650101 19.93766327   43.9080478576 36.5055646443 32.4777167198
 43.7880786668 20.8244147577 31.3566513412 12.8496919548 36.3209076612]
Mean: 31.736800188350863
Standard deviation: 10.057620103087322


In [156]:
f_i=0
l_i=10
some_dat =X_test.iloc[f_i:l_i]
#print(some_dat)
some_label = y_test.iloc[f_i:l_i]

print("Predictions:", tree_reg.predict(some_dat))
print("Labels:", list(some_label))


Predictions: [   0.    0.    0.  -70.  -20.    0.   65.    0. -135.    0.]
Labels: [0.0, 0.0, 0.0, -50.0, 0.0, 0.0, 110.0, 0.0, -140.0, 0.0]


In [157]:
#OTHER MODEL ->  RandomForestRegressor 
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(train, train_labels)

predictions = forest_reg.predict(train)
forest_mse = mean_squared_error(train_labels,predictions)
forest_rmse = np.sqrt(forest_mse)
print(forest_rmse)#


forest_scores = cross_val_score(forest_reg,train,train_labels,
                    scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)


display_scores(forest_rmse_scores) #Mean: 50182.303100336096,Standard deviation: 2097.0810550985693

8.794846010827401
Scores: [33.509257474  19.6118440245 26.5082648865 21.9446523478 22.9448124357
 28.8540262391 17.3498273286 21.7235896406 13.8579283865 18.8938641385]
Mean: 22.519806690190364
Standard deviation: 5.4891172987822525


In [158]:
f_i=200
l_i=210
some_data = train.iloc[f_i:l_i]
some_labels = train_labels.iloc[f_i:l_i]

print("Predictions:", forest_reg.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [86.65  0.    0.    0.    0.    3.45  0.    0.   44.21 99.5 ]
Labels: [85.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 45.0, 170.0]


In [160]:
''' Evaluate Your System on the Test Set- RandomForestRegressor '''
from sklearn.metrics import mean_squared_error
y_test = test_set["correction"].copy()
X_test = test_set.drop("correction", axis=1)

f_i=20
l_i=21
final_predictions =  forest_reg.predict(X_test.iloc[f_i:l_i])
print("Predictions test set:",final_predictions )
print("Labels test set:", list(y_test.iloc[f_i:l_i]))


Predictions test set: [-65.1]
Labels test set: [-80.0]


In [182]:
### test with real data
import csv
class TEST:
    def __init__(self):
        self.a=315
        self.b=325
        self.c=335
        self.d=345
        self.e=355
        self.f=5
        self.g=15
        self.h=25
        self.i=35
        self.k=45
        self.PATH_TEST="C:/Users/Owner/Desktop/projects\ML/chapter_2/one_scan.csv"
        
    def create_test_file(self):
        with open(self.PATH_TEST,'w',newline='') as new_file:
                    writer = csv.writer(new_file)
                    fields = [f"{self.b}", f"{self.c}", f"{self.d}",f"{self.e}",f"{self.f}",f"{self.g}",f"{self.h}",f"{self.i}",f"{self.k}",'direction']
                    writer.writerow(fields)
                    writer.writerow([30,26.5,22.5,22,21.5,22.5,23,25,40,345])  # 
        return pd.read_csv(self.PATH_TEST)
t=TEST()    
test=t.create_test_file()
print(test)
            

   325   335   345  355     5    15  25  35  45  direction
0   30  26.5  22.5   22  21.5  22.5  23  25  40        345


In [178]:
# test=[[300,300,290,280,200,200,1000,1200,1234,20]]
print("Predictions DecisionTreeRegressor :", tree_reg.predict(test))
print("Predictions RandomForestRegressor :" ,forest_reg.predict(test))

Predictions DecisionTreeRegressor : [-155.]
Predictions RandomForestRegressor : [-146.95]


In [185]:
### SAVE MODELS 
import joblib
#joblib.dump(forest_reg, "my_model.pkl")
# and later...
forest_reg_loaded = joblib.load("my_model.pkl")

In [186]:
print("Predictions RandomForestRegressor :" ,forest_reg_loaded.predict(test))

Predictions RandomForestRegressor : [-146.95]
